In [59]:
import codecs,unidecode,yaml,glob,os,shutil,re
import itertools as it
import pandas as pd
from cellbell import ding

In [60]:
nQuizz=10
# fEtudiants=u"/Users/gilles/Desktop/19-Morpho1-Quizz.csv"
# fEtudiantsNew=u"/Users/gilles/Desktop/19-LNS2U2-Liste.csv"
# fQuizzoodle=u"/Users/gilles/Desktop/Quizz%02d.csv"%nQuizz
# fAliasQDLE=u"/Users/gilles/Desktop/Quizz-Alias.yaml"
fMajeure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Majeure.csv"
fMineure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Mineure.csv"
fCM=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv"
fAliasQDLE=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-Alias.yaml"
fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz%02d.csv"%nQuizz
fNotes=fQuizzoodle.replace(u".csv",u"-Notes.csv")

In [61]:
oldNotes={}
if os.path.isfile(fNotes):
    with codecs.open(fNotes,"r",encoding="utf8") as inFile:
        lNotes=inFile.readlines()
    for l in lNotes[1:]:
        lEls=l.strip().split("\t")
        oldNotes["\t".join(lEls[:2])]=int(lEls[2])
    fNotesOld=fNotes.replace(u"-Notes.csv",u"-Notes-old.csv")
    if not os.path.isfile(fNotesOld):
        shutil.copyfile(fNotes,fNotesOld)
oldNotes

{u'ABDALLAH\tMAELIS': 12,
 u'ABDOU\tMELOE': 10,
 u'ABRIEU\tPAULINE': 17,
 u'ALEXANDRU\tJULIE': 15,
 u'ALIZON\tMANON': 17,
 u'ANDRIES\tSOPHIE': 12,
 u'ARDOIN\tFLORIAN': 13,
 u'ASSELIN\tISANDRE': 17,
 u'AUDRAN\tMANON': 15,
 u'BAGUR\tTIM': 10,
 u'BANLIAT\tADAM': 13,
 u'BARRAT\tFANY': 17,
 u'BEAUVAIS\tCAMILLE': 15,
 u'BECCACCIA\tANDREA': 15,
 u'BENBRAHIM\tMENEL': 12,
 u'BENMOUMENE\tKENZA': 13,
 u'BERARD\tLOUIS': 17,
 u'BESSE-RUUD\tNINA': 13,
 u'BLANDIE\tMELISSA': 15,
 u'BORDERIE\tMAEVA': 12,
 u'BOTTIN\tANDREA': 15,
 u'BOUE\tALICE': 10,
 u'BOURDET\tCHIFIU': 13,
 u'BRENDEL\tTIMOTHEE': 13,
 u'BUELENS\tLAETITIA': 12,
 u'CAMARA\tSINAI': 12,
 u'CANTIN\tLILAS-ROSE': 13,
 u'CAPARROS\tLEA': 15,
 u'CARAES\tSALOME': 12,
 u'CARNEVALE\tHUGO': 17,
 u'CAZETTE\tJULIE': 13,
 u'CHALUS\tKATY': 10,
 u'CHASSIN\tLAURA': 13,
 u'COMAS\tMICKAEL': 12,
 u'CORDETTE\tFLORIANE': 10,
 u'COULAIS\tIRIS': 17,
 u'COURTIE\tANAIS': 12,
 u'COUTAND\tALEXANDRA': 12,
 u'DE CARVALHO\tNATHAN': 15,
 u'DEJEANS\tLUCAS': 13,
 u'DELAFLO

In [62]:
sep=u"\t"
nbQuestionQuizz={i:(7 if i<7 else 6) for i in range(1,nQuizz+1)}
nbQuestionQuizz

{1: 7, 2: 7, 3: 7, 4: 7, 5: 7, 6: 7, 7: 6, 8: 6, 9: 6, 10: 6}

In [63]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inFile:
        correspondances=yaml.safe_load(inFile)
    alias=correspondances["alias2etu"]
except:
    alias={}
alias

{'001lherfouf': ['lherfouf001'],
 '04247454': [],
 '138': [],
 '28150417': [],
 '38 27 56': ['eherrera001'],
 '380908': ['bsalman001'],
 '390305': ['lmagit001'],
 '49814523': [],
 '56416920': [],
 '70207082': [],
 '777': [],
 '7925871': ['?'],
 '92317548': [],
 'a.cornillier': ['acornilli001'],
 'aanny': ['?'],
 'abailly003': ['abailly003'],
 'abeccacci001': ['abeccacci001'],
 'abertrand014': ['?'],
 'abottin001': ['abottin001'],
 'acornillier': ['acornilli001'],
 'acourrege002': ['acourrege002'],
 'ada': ['?'],
 'adartigue': ['adartigue004'],
 'adeline pacot': ['apacot001'],
 'adeniaud002': ['adeniaud002'],
 'adiarra001 + cbeauvais004': ['adiarra001', 'cbeauvais004'],
 'adinne001': ['adinne001'],
 'adinne001 bowicz001': ['adinne001', 'pzubowicz001'],
 'adinne001 lepinoux001': ['adinne001', 'lepinoux001'],
 'adutour001': ['adutour001'],
 'aguichon002': ['aguichon002'],
 'alabinal001': ['alabinal001'],
 'alauret005': ['alauret005'],
 'amachefer001': ['amachefer001'],
 'amariehe001': ['a

In [36]:
try:
    dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8")
    dfCM.set_index(u"Identifiant",inplace=True)
except:
    dfMajeure=pd.read_csv(filepath_or_buffer=fMajeure,sep="\t",encoding="utf8")
    dfMajeure.set_index(u"Identifiant",inplace=True)
    dMajeure=dfMajeure[u"Code étudiant"].to_dict()
    dMajeure

    dfMineure=pd.read_csv(filepath_or_buffer=fMineure,sep="\t",encoding="utf8")
    dfMineure.set_index(u"Identifiant",inplace=True)
    dMineure=dfMineure[u"Code étudiant"].to_dict()
    dMineure

    dfCM=dfMajeure.append(dfMineure,sort=False)
    dfCM.to_csv(path_or_buf=fCM,sep="\t",encoding="utf8")
dfCM[u"Code étudiant"]=dfCM[u"Code étudiant"].astype(unicode)
dfCM

,Nom patronymique,Nom usuel,Prénom,Code étudiant,Mail,Note
Identifiant,,,,,,
mabdallah008,ABDALLAH,NaN,MAELIS,385180,maelis.abdallah@etu.u-bordeaux-montaigne.fr,NaN
cabderreb001,ABDERREBI,NaN,CAMILLIA,374204,camillia.abderrebi@etu.u-bordeaux-montaigne.fr,NaN
mabdou005,ABDOU,NaN,MELOE,384307,meloe.abdou@etu.u-bordeaux-montaigne.fr,NaN
nandriant001,ANDRIANTSIFERANA,NaN,NANDRIANINA,384656,nandrianina.andriantsiferana@etu.u-bordeaux-mo...,NaN
sandries001,ANDRIES,NaN,SOPHIE,380354,sophie.andries@etu.u-bordeaux-montaigne.fr,NaN
laranaz001,ARANAZ,NaN,LOUIS,385151,louis.aranaz@etu.u-bordeaux-montaigne.fr,NaN
iasselin001,ASSELIN,NaN,ISANDRE,385107,isandre.asselin@etu.u-bordeaux-montaigne.fr,NaN
maudran001,AUDRAN,NaN,MANON,380271,manon.audran@etu.u-bordeaux-montaigne.fr,NaN
aazouni002,AZOUNI,NaN,ANAIS,387238,anais.azouni@etu.u-bordeaux-montaigne.fr,NaN


In [37]:
dfCM[dfCM.index.str.contains("laugt")]

,Nom patronymique,Nom usuel,Prénom,Code étudiant,Mail,Note
Identifiant,,,,,,
jlaugt002,LAUGT,NaN,JULIE,381437,julie.laugt@etu.u-bordeaux-montaigne.fr,NaN


In [38]:
def chercherIdentifiant(chaine):
    print chaine
    codes=[]
    identifiants=[]
    nobody=False
    if chaine in alias:
        codes=[dfCM.loc[a,u"Code étudiant"] for a in alias[chaine] if a in dfCM.index]       
    else:
        while not codes and not nobody:
            chaineDecode=unidecode.unidecode(chaine).lower()
            saisie=raw_input(u"quels identifiants pour %s"%chaineDecode)
            saisie = unidecode.unidecode(saisie).lower()
            if saisie==u"?":
                identifiants=[u"?"]
                codes=[]
                nobody=True
                print codes or nobody
            elif u" " in saisie:
                identifiants=saisie.split(u" ")
                if not all((a in dfCM.index) or (a==u"?") for a in identifiants):
                    identifiants=[]
                    codes=[]
                else:
                    identifiants=[a if a in dfCM.index else u"?" for a in identifiants]
                    codes=[dfCM.loc[a,u"Code étudiant"] for a in identifiants if a in dfCM.index]
                    print identifiants,codes
            elif saisie in dfCM.index:
                identifiants=[saisie]
                codes=[dfCM.loc[saisie,u"Code étudiant"]]
    alias[chaine]=identifiants
    return codes

def decomposerNom(nom):
    result=[]
    m=re.findall(ur"(\w{5,9}\d{3})",nom)
    if m:
        for e in m:
            if e in dfCM.index:
                result.append(dfCM.loc[e,u"Code étudiant"])
            else:
                result.extend(chercherIdentifiant(e))
    else:
        result.extend(chercherIdentifiant(nom))
    return result


In [39]:
print "flo26" in alias
[dfCM.loc[a,u"Code étudiant"] for a in alias["flo26"] if a in dfCM.index]


True


[u'382471']

In [40]:
with codecs.open(fQuizzoodle,"r",encoding="utf8") as input:
    lQuizz=input.readlines()
    
nbBons={}
ansTimes={}
for ligne in lQuizz[1:]:
    if ligne.strip()!="":
        splits=ligne.strip().replace("&amp;","&").split(";")
        splits=ligne.strip().replace("&amp;","&").split(";")
        lenNoms=len(splits)-2
        nom=" ".join(splits[:lenNoms])
        bons=splits[lenNoms]
        temps=splits[lenNoms+1]
        nom=nom.strip().lower()
        if nom in dfCM.index:
            akas=[dfCM.loc[nom,u"Code étudiant"]]
        elif nom in alias:
            akas=[dfCM.loc[a,u"Code étudiant"] for a in alias[nom] if a in dfCM.index]
        else:
            akas=chercherIdentifiant(nom)
            alias[nom]=akas
#        print nom,akas,bons
        for aka in akas:
            if aka in nbBons:
                nbBons[aka]+=int(bons)
            else:
                nbBons[aka]=int(bons)
            if aka in ansTimes:
                ansTimes[aka]+=float(temps)
            else:
                ansTimes[aka]=float(temps)

#notes={aka:(round(float(nbBons[aka])/6*10+10) for aka in nbBons)}
ansTimes

{u'348881': 167.42,
 u'350181': 226.54,
 u'350705': 207.11,
 u'351139': 120.11,
 u'351949': 238.98,
 u'365398': 156.44,
 u'366831': 146.13,
 u'368148': 196.08,
 u'368340': 147.19,
 u'370402': 243.24,
 u'371250': 226.7,
 u'371638': 160.08,
 u'371845': 197.83,
 u'372039': 202.88,
 u'373134': 222.23,
 u'378687': 77.12,
 u'380010': 222.23,
 u'380103': 201.93,
 u'380123': 0.0,
 u'380150': 209.86,
 u'380187': 207.28,
 u'380189': 187.07,
 u'380271': 183.81,
 u'380282': 86.53,
 u'380308': 232.62,
 u'380316': 226.54,
 u'380354': 187.99,
 u'380366': 217.15,
 u'380427': 110.91,
 u'380475': 135.13,
 u'380482': 249.6,
 u'380506': 171.95,
 u'380627': 213.76,
 u'380645': 243.01,
 u'380685': 71.73,
 u'380705': 186.98,
 u'380795': 133.09,
 u'380846': 124.76,
 u'380853': 64.07,
 u'380991': 182.1,
 u'381009': 237.0,
 u'381077': 209.86,
 u'381097': 219.75,
 u'381195': 162.3,
 u'381209': 215.61,
 u'381228': 99.59,
 u'381305': 147.87,
 u'381340': 227.27,
 u'381350': 98.85,
 u'381351': 177.06,
 u'381358': 19

In [41]:
dfCM#.set_index("Identifiant")
# for id in nbBons:
#     dfCM.loc[id,"note"]=nbBons[id]

,Nom patronymique,Nom usuel,Prénom,Code étudiant,Mail,Note
Identifiant,,,,,,
mabdallah008,ABDALLAH,NaN,MAELIS,385180,maelis.abdallah@etu.u-bordeaux-montaigne.fr,NaN
cabderreb001,ABDERREBI,NaN,CAMILLIA,374204,camillia.abderrebi@etu.u-bordeaux-montaigne.fr,NaN
mabdou005,ABDOU,NaN,MELOE,384307,meloe.abdou@etu.u-bordeaux-montaigne.fr,NaN
nandriant001,ANDRIANTSIFERANA,NaN,NANDRIANINA,384656,nandrianina.andriantsiferana@etu.u-bordeaux-mo...,NaN
sandries001,ANDRIES,NaN,SOPHIE,380354,sophie.andries@etu.u-bordeaux-montaigne.fr,NaN
laranaz001,ARANAZ,NaN,LOUIS,385151,louis.aranaz@etu.u-bordeaux-montaigne.fr,NaN
iasselin001,ASSELIN,NaN,ISANDRE,385107,isandre.asselin@etu.u-bordeaux-montaigne.fr,NaN
maudran001,AUDRAN,NaN,MANON,380271,manon.audran@etu.u-bordeaux-montaigne.fr,NaN
aazouni002,AZOUNI,NaN,ANAIS,387238,anais.azouni@etu.u-bordeaux-montaigne.fr,NaN


In [42]:
nbQuestions=nbQuestionQuizz[nQuizz]
def noteEtu(etus):
    notes=[]
    for etu in etus:
        note=int(round(float(nbBons[etu])/nbQuestions*10+10))
        if note==10 and ansTimes[etu]<10:
            note=8.0
        print etu,note
        notes.append(note)
    return notes

In [43]:
print len(nbBons)
ansTimes

145


{u'348881': 167.42,
 u'350181': 226.54,
 u'350705': 207.11,
 u'351139': 120.11,
 u'351949': 238.98,
 u'365398': 156.44,
 u'366831': 146.13,
 u'368148': 196.08,
 u'368340': 147.19,
 u'370402': 243.24,
 u'371250': 226.7,
 u'371638': 160.08,
 u'371845': 197.83,
 u'372039': 202.88,
 u'373134': 222.23,
 u'378687': 77.12,
 u'380010': 222.23,
 u'380103': 201.93,
 u'380123': 0.0,
 u'380150': 209.86,
 u'380187': 207.28,
 u'380189': 187.07,
 u'380271': 183.81,
 u'380282': 86.53,
 u'380308': 232.62,
 u'380316': 226.54,
 u'380354': 187.99,
 u'380366': 217.15,
 u'380427': 110.91,
 u'380475': 135.13,
 u'380482': 249.6,
 u'380506': 171.95,
 u'380627': 213.76,
 u'380645': 243.01,
 u'380685': 71.73,
 u'380705': 186.98,
 u'380795': 133.09,
 u'380846': 124.76,
 u'380853': 64.07,
 u'380991': 182.1,
 u'381009': 237.0,
 u'381077': 209.86,
 u'381097': 219.75,
 u'381195': 162.3,
 u'381209': 215.61,
 u'381228': 99.59,
 u'381305': 147.87,
 u'381340': 227.27,
 u'381350': 98.85,
 u'381351': 177.06,
 u'381358': 19

In [44]:
print "Quizz",nQuizz
with codecs.open(fNotes,"w",encoding="utf8") as output:
    output.write(u"%s\t%s\t%s\t%s"%("NOM\tPRENOM","NOTE","TEMPS","ID")+"\n")
    for etu in sorted(nbBons):
        print etu
        idEtu=dfCM[dfCM[u"Code étudiant"]==etu].index.values[0]
        note=int(round(float(nbBons[etu])/nbQuestions*10+10))
        if note==10 and ansTimes[etu]<10:
            note=8.0
        if idEtu in dfCM.index:
            num=etu
            nomEtu=dfCM.loc[idEtu,u"Nom patronymique"]
            prenomEtu=dfCM.loc[idEtu,u"Prénom"]
            nom=u"%s\t%s"%(nomEtu,prenomEtu)
            timeEtu=ansTimes[etu]
        else:
            num="???"
        if etu=="?": 
            nom="?\t?"
#         else:
#             nom=etu
        if not etu in oldNotes or oldNotes[etu]!=int(note):
            print u"%s\t%d\t%.2f\t%s"%(nom,note,timeEtu,etu)
        output.write(u'%s\t%d\t%.2f\t%s'%(nom,note,timeEtu,num)+"\n")

Quizz 10
348881
POULAIN	CELINE	13	167.42	348881
350181
NAPRIX	NORA	10	226.54	350181
350705
BERARD	LOUIS	17	207.11	350705
351139
GUINEE	GABRIELLE	15	120.11	351139
351949
MAMI	ELISHEVAH	10	238.98	351949
365398
GIRARD	EMILIE	17	156.44	365398
366831
DESHAYES	ADELE	13	146.13	366831
368148
LEWIS	ALEXIE	13	196.08	368148
368340
SAPUTA	JONATHAN	12	147.19	368340
370402
GAZARIAN	DAVID	13	243.24	370402
371250
JAMME	LUCAS	12	226.70	371250
371638
PETITBERGHIEN	MARION	17	160.08	371638
371845
OLENGA-DIARRA	ROSELYNE	13	197.83	371845
372039
MURAT	MAUDE	15	202.88	372039
373134
SCHOONBEEK	nan	13	222.23	373134
378687
FISCHER	ILLONA	15	77.12	378687
380010
BESSE-RUUD	NINA	13	222.23	380010
380103
DUPEBE	MAYLIS	13	201.93	380103
380123
MAROLLEAU	LOU	8	0.00	380123
380150
OUAFI	AMELE	12	209.86	380150
380187
STASICA	ALEX	12	207.28	380187
380189
CHASSIN	LAURA	13	187.07	380189
380271
AUDRAN	MANON	15	183.81	380271
380282
DUVAL	CLEMENT	13	86.53	380282
380308
LESPIAU	LEILA	12	232.62	380308
380316
CORDETTE	FLORIANE	10	2

In [45]:
dfCM[dfCM.index.str.contains("fle")]

,Nom patronymique,Nom usuel,Prénom,Code étudiant,Mail,Note
Identifiant,,,,,,
flecoeur001,LECOEUR,NaN,FLORIAN,382471,florian.lecoeur@etu.u-bordeaux-montaigne.fr,NaN


In [46]:
dfCM[dfCM[u"Code étudiant"]=="385180"].index.values[0]

u'mabdallah008'

In [68]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

In [69]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'001lherfouf': ['lherfouf001'],
  '04247454': [],
  '138': [],
  '28150417': [],
  '38 27 56': ['eherrera001'],
  '380908': ['bsalman001'],
  '390305': ['lmagit001'],
  '49814523': [],
  '56416920': [],
  '70207082': [],
  '777': [],
  '7925871': ['?'],
  '92317548': [],
  'a.cornillier': ['acornilli001'],
  'aanny': ['?'],
  'abailly003': ['abailly003'],
  'abeccacci001': ['abeccacci001'],
  'abertrand014': ['?'],
  'abottin001': ['abottin001'],
  'acornillier': ['acornilli001'],
  'acourrege002': ['acourrege002'],
  'ada': ['?'],
  'adartigue': ['adartigue004'],
  'adeline pacot': ['apacot001'],
  'adeniaud002': ['adeniaud002'],
  'adiarra001 + cbeauvais004': ['adiarra001', 'cbeauvais004'],
  'adinne001': ['adinne001'],
  'adinne001 bowicz001': ['adinne001', 'pzubowicz001'],
  'adinne001 lepinoux001': ['adinne001', 'lepinoux001'],
  'adutour001': ['adutour001'],
  'aguichon002': ['aguichon002'],
  'alabinal001': ['alabinal001'],
  'alauret005': ['alauret005'],
  'amach

In [70]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

In [71]:
ding()

# FIN

In [64]:
newAlias={}
for k,v in alias.iteritems():
    if v:
        newV=[]
        for e in v:
            m=re.match(ur"\d+",e)
            if m:
                idEtu=dfCM[dfCM[u"Code étudiant"]==e].index.values[0]
                print k,idEtu
                newV.append(idEtu)
            else:
                newV.append(e)
        newAlias[k]=newV
    else:
        newAlias[k]=[]
newAlias

{'001lherfouf': ['lherfouf001'],
 '04247454': [],
 '138': [],
 '28150417': [],
 '38 27 56': ['eherrera001'],
 '380908': ['bsalman001'],
 '390305': ['lmagit001'],
 '49814523': [],
 '56416920': [],
 '70207082': [],
 '777': [],
 '7925871': ['?'],
 '92317548': [],
 'a.cornillier': ['acornilli001'],
 'aanny': ['?'],
 'abailly003': ['abailly003'],
 'abeccacci001': ['abeccacci001'],
 'abertrand014': ['?'],
 'abottin001': ['abottin001'],
 'acornillier': ['acornilli001'],
 'acourrege002': ['acourrege002'],
 'ada': ['?'],
 'adartigue': ['adartigue004'],
 'adeline pacot': ['apacot001'],
 'adeniaud002': ['adeniaud002'],
 'adiarra001 + cbeauvais004': ['adiarra001', 'cbeauvais004'],
 'adinne001': ['adinne001'],
 'adinne001 bowicz001': ['adinne001', 'pzubowicz001'],
 'adinne001 lepinoux001': ['adinne001', 'lepinoux001'],
 'adutour001': ['adutour001'],
 'aguichon002': ['aguichon002'],
 'alabinal001': ['alabinal001'],
 'alauret005': ['alauret005'],
 'amachefer001': ['amachefer001'],
 'amariehe001': ['a

In [65]:
for k,v in newAlias.iteritems():
    print k,v
    print alias[k]
    print

lenocq ['lenocq001']
['lenocq001']

lhuet003 ['lhuet003']
['lhuet003']

jtm gile []
[]

mroudaut003 ['mroudaut003']
['mroudaut003']

marion []
[]

thuyngannguyendao ['qnguyen004']
['qnguyen004']

francesca ['fnetry001']
['fnetry001']

lprovost00 ['lprovost002']
['lprovost002']

kchalus001 ['kchalus001']
['kchalus001']

kate ['?']
['?']

eguillot004,jmocaer001 ['eguillot004', 'jmocaer001']
['eguillot004', 'jmocaer001']

jlacombe008 / lmarollea002 ['jlacombe008', 'lmarollea002']
['jlacombe008', 'lmarollea002']

llandreau002 ['llandreau002']
['llandreau002']

nora ['ngillet002']
['ngillet002']

teo_guillaut ['tguillaut001']
['tguillaut001']

vmouchoux ['vmouchoux001']
['vmouchoux001']

okamari no suzuki []
[]

le plus mignon ['?']
['?']

anihouarn001/jlaugt002 ['jlaugt002', 'anihouarn001']
['jlaugt002', 'anihouarn001']

amy rodriguez ['arodrigue025']
['arodrigue025']

lmarolleau ['lmarollea002']
['lmarollea002']

hattie chatt ['?']
['?']

sandro teixeira ['steixeira003']
['steixeira003']


kborde001 ['kborde001']
['kborde001']

rouard manon ['mrouard001']
['mrouard001']

kndjadi ['kndjadi001']
['kndjadi001']

edupontta001 ['edupontta001']
['edupontta001']

léo []
[]

pabrieu001 ['pabrieu001']
['pabrieu001']

jbenesy001,eroofthoo001 ['jbenesy001', 'eroofthoo001']
['jbenesy001', 'eroofthoo001']

soraya []
[]

bonade001 ['ebonade001']
['ebonade001']

mgibaru001 , icoulais001 ['mgibaru001', 'icoulais001']
['mgibaru001', 'icoulais001']

le fils de gillou []
[]

lherfouf001 ['lherfouf001']
['lherfouf001']

guillaut, monnet ['tguillaut001', 'mmonnet002']
['tguillaut001', 'mmonnet002']

hchatt001 []
[]

lucile de la fuente ['ldelafuen001']
['ldelafuen001']

andrea []
[]

tgodard002, amele002 ['tgodard002']
['tgodard002']

jmetais001 ['jmetais001']
['jmetais001']

heye []
[]

kshigetom001 ['kshigetom001']
['kshigetom001']

léonore dinety ['ldinety001']
['ldinety001']

jeanne3 []
[]

msarci001 ['msarci001']
['msarci001']

abertrand014 ['?']
['?']

nnaprix001-fcordette001 ['nnaprix

In [66]:
alias=newAlias

In [67]:
alias

{'001lherfouf': ['lherfouf001'],
 '04247454': [],
 '138': [],
 '28150417': [],
 '38 27 56': ['eherrera001'],
 '380908': ['bsalman001'],
 '390305': ['lmagit001'],
 '49814523': [],
 '56416920': [],
 '70207082': [],
 '777': [],
 '7925871': ['?'],
 '92317548': [],
 'a.cornillier': ['acornilli001'],
 'aanny': ['?'],
 'abailly003': ['abailly003'],
 'abeccacci001': ['abeccacci001'],
 'abertrand014': ['?'],
 'abottin001': ['abottin001'],
 'acornillier': ['acornilli001'],
 'acourrege002': ['acourrege002'],
 'ada': ['?'],
 'adartigue': ['adartigue004'],
 'adeline pacot': ['apacot001'],
 'adeniaud002': ['adeniaud002'],
 'adiarra001 + cbeauvais004': ['adiarra001', 'cbeauvais004'],
 'adinne001': ['adinne001'],
 'adinne001 bowicz001': ['adinne001', 'pzubowicz001'],
 'adinne001 lepinoux001': ['adinne001', 'lepinoux001'],
 'adutour001': ['adutour001'],
 'aguichon002': ['aguichon002'],
 'alabinal001': ['alabinal001'],
 'alauret005': ['alauret005'],
 'amachefer001': ['amachefer001'],
 'amariehe001': ['a

# Chercher les noms des étudiants